# Malware report analysis

Identifying salient topics and terms in the APT reports.

In [1]:
from __future__ import print_function

In [2]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import cPickle

## Load the pickled APT reports


In [6]:
apt_reports = cPickle.load(open("APTreports_no.p", "rb"))
print(len(apt_reports))

284


## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or TF-IDF form.

In [11]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(apt_reports)
print(dtm_tf.shape)

(284, 4574)


In [12]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(apt_reports)
print(dtm_tfidf.shape)

(284, 4574)


## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted.

In [13]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## Visualzing the models with pyLDAvis

In [14]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      18.697710        1       1 -0.062920 -0.091984
14     16.203213        1       2  0.024268 -0.039897
4      13.382854        1       3 -0.072290 -0.077790
16     13.354474        1       4  0.095248  0.024155
19      7.865188        1       5  0.185461  0.115760
18      7.407782        1       6  0.176645  0.095245
13      4.742795        1       7 -0.327801  0.363876
3       3.957775        1       8 -0.067193 -0.081048
1       3.907358        1       9 -0.135752 -0.109874
15      2.640126        1      10 -0.076411 -0.094025
0       2.070720        1      11  0.205542  0.165000
12      2.050365        1      12 -0.038895 -0.069692
10      1.471526        1      13 -0.055041 -0.060708
11      1.044824        1      14 -0.045655 -0.056411
17      0.955053        1      15  0.042482  0.018649
5       0.236017        1      16 -0.010634 -0.027849
6       0.003055        1      17  0.047030 -0.013366
7       0.003055        1      18  0.035785 -0.022783
8       0.003055        1      19  0.038985 -0.019599
2       0.003055        1      20  0.041145 -0.017657, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
608   Default  18798.000000           cid  18798.000000  30.0000  30.0000
219   Default   1187.000000           apt   1187.000000  29.0000  29.0000
2258  Default   1136.000000     kaspersky   1136.000000  28.0000  28.0000
595   Default   2316.000000         china   2316.000000  27.0000  27.0000
826   Default    936.000000       contact    936.000000  26.0000  26.0000
507   Default   1009.000000     campaigns   1009.000000  25.0000  25.0000
2140  Default    685.000000  intelreports    685.000000  24.0000  24.0000
3593  Default    622.000000       russian    622.000000  23.0000  23.0000
1631  Default    650.000000       fireeye    650.000000  22.0000  22.0000
1608  Default   1811.000000        figure   1811.000000  21.0000  21.0000
3967  Default    545.000000       stuxnet    545.000000  20.0000  20.0000
545   Default    411.000000        center    411.000000  19.0000  19.0000
1275  Default    769.000000          duqu    769.000000  18.0000  18.0000
4176  Default    614.000000           tlp    614.000000  17.0000  17.0000
597   Default   1443.000000       chinese   1443.000000  16.0000  16.0000
2018  Default    365.000000      incident    365.000000  15.0000  15.0000
2583  Default   1296.000000      military   1296.000000  14.0000  14.0000
656   Default    393.000000       cluster    393.000000  13.0000  13.0000
2250  Default    472.000000          july    472.000000  12.0000  12.0000
4456  Default    908.000000       warfare    908.000000  11.0000  11.0000
2072  Default    817.000000          info    817.000000  10.0000  10.0000
3299  Default    834.000000           rat    834.000000   9.0000   9.0000
3019  Default    721.000000           pla    721.000000   8.0000   8.0000
3398  Default    803.000000      registry    803.000000   7.0000   7.0000
661   Default    373.000000           cnc    373.000000   6.0000   6.0000
1260  Default    676.000000        driver    676.000000   5.0000   5.0000
964   Default    912.000000           cve    912.000000   4.0000   4.0000
1935  Default    792.000000       hotmail    792.000000   3.0000   3.0000
431   Default    341.000000        botnet    341.000000   2.0000   2.0000
3524  Default    593.000000     retrieved    593.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
410   Topic20      0.005570        blocks    126.666906   0.3642  -7.8384
2811  Topic20      0.006025           oil    177.126025   0.1073  -7.7600
1141  Topic20      0.006655        device    310.269436  -0.3538  -7.6605
2460  Topic20      0.008234          main    817.791391  -1.1100  -7.4476
3919  Topic20      0.005978    

In [15]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x             y
topic                                                    
16     92.940018        1       1  0.068887 -7.275424e-08
13      0.371578        1       2 -0.003835 -5.738578e-05
15      0.371578        1       3 -0.003755 -3.602477e-05
9       0.371578        1       4 -0.000885  5.357973e-04
18      0.371578        1       5  0.000697 -2.245487e-04
17      0.371578        1       6 -0.005170  1.402091e-05
12      0.371578        1       7 -0.002164 -1.586479e-04
10      0.371578        1       8 -0.003125 -3.350899e-05
11      0.371578        1       9 -0.004366  1.033469e-06
4       0.371578        1      10 -0.001914 -4.684558e-05
14      0.371578        1      11 -0.002774 -1.513535e-05
5       0.371578        1      12 -0.002898 -3.768431e-05
6       0.371578        1      13 -0.004686  1.877962e-06
8       0.371578        1      14 -0.004298 -3.732735e-05
1       0.371578        1      15 -0.004308  2.721192e-06
3       0.371578        1      16 -0.005095  1.009278e-05
19      0.371578        1      17 -0.004733  7.302541e-06
7       0.371578        1      18 -0.005182  3.607057e-05
0       0.371578        1      19 -0.004894  2.004906e-05
2       0.371578        1      20 -0.005502  1.821574e-05, topic_info=     Category      Freq             Term     Total  loglift  logprob
term                                                                
595   Default  9.000000            china  9.000000  30.0000  30.0000
1608  Default  8.000000           figure  8.000000  29.0000  29.0000
219   Default  7.000000              apt  7.000000  28.0000  28.0000
597   Default  6.000000          chinese  6.000000  27.0000  27.0000
3003  Default  6.000000              php  6.000000  26.0000  26.0000
2258  Default  5.000000        kaspersky  5.000000  25.0000  25.0000
2583  Default  5.000000         military  5.000000  24.0000  24.0000
2575  Default  5.000000            micro  5.000000  23.0000  23.0000
4456  Default  5.000000          warfare  5.000000  22.0000  22.0000
964   Default  5.000000              cve  5.000000  21.0000  21.0000
777   Default  5.000000    configuration  5.000000  20.0000  20.0000
1631  Default  4.000000          fireeye  4.000000  19.0000  19.0000
3019  Default  5.000000              pla  5.000000  18.0000  18.0000
507   Default  4.000000        campaigns  4.000000  17.0000  17.0000
4176  Default  4.000000              tlp  4.000000  16.0000  16.0000
3299  Default  4.000000              rat  4.000000  15.0000  15.0000
2140  Default  4.000000     intelreports  4.000000  14.0000  14.0000
3393  Default  4.000000       registrant  4.000000  13.0000  13.0000
705   Default  4.000000        companies  4.000000  12.0000  12.0000
1224  Default  4.000000              doc  4.000000  11.0000  11.0000
4233  Default  4.000000            trend  4.000000  10.0000  10.0000
4043  Default  4.000000         symantec  4.000000   9.0000   9.0000
2632  Default  4.000000           module  4.000000   8.0000   8.0000
2501  Default  4.000000            march  4.000000   7.0000   7.0000
2998  Default  4.000000         phishing  4.000000   6.0000   6.0000
1265  Default  4.000000          dropper  4.000000   5.0000   5.0000
109   Default  4.000000            agent  4.000000   4.0000   4.0000
3840  Default  4.000000            spear  4.000000   3.0000   3.0000
1275  Default  3.000000             duqu  3.000000   2.0000   2.0000
3883  Default  3.000000            stage  3.000000   1.0000   1.0000
...       ...       ...              ...       ...      ...      ...
3161  Topic20  0.003239        procedure  0.600795   0.3723  -8.2931
3418  Topic20  0.003127         reliable  0.398505   0.7476  -8.3284
2411  Topic20  0.003083          locally  0.330199   0.9214  -8.3425
29    Topic20  0.003235         accuracy  0.668518   0.2640  -8.2946
1535  Topic20  0.003205         extended  0.576161   0.4036  -8.3036
1282  Topic20  0.003329         dynamics  1.161759  -0.2598

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      12.691766        1       1  0.193674 -0.012044
19     11.686348        1       2  0.190780  0.094852
8       8.568093        1       3 -0.131908  0.178354
15      7.682942        1       4 -0.154770  0.001473
14      7.401868        1       5  0.068408 -0.447076
0       6.705742        1       6  0.007767 -0.323911
13      6.569009        1       7 -0.220604 -0.207815
4       5.887326        1       8  0.047581 -0.154507
10      4.939733        1       9  0.095683  0.211729
16      4.438698        1      10  0.257604 -0.339387
5       4.292685        1      11  0.381606 -0.171807
1       4.085641        1      12 -0.021239  0.462675
2       3.037200        1      13  0.363107  0.219992
9       2.451119        1      14 -0.379412  0.253809
17      2.436387        1      15  0.220519  0.380356
7       1.891398        1      16 -0.207634  0.400122
3       1.511184        1      17 -0.470146 -0.164653
11      1.445251        1      18 -0.473191  0.051133
12      1.346250        1      19  0.507607  0.017560
18      0.931361        1      20 -0.275431 -0.450858, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
5017  Default  4649.000000         max  4649.000000  30.0000  30.0000
2653  Default  2384.000000         edu  2384.000000  29.0000  29.0000
3524  Default  2037.000000         god  2037.000000  28.0000  28.0000
7678  Default  1274.000000       space  1274.000000  27.0000  27.0000
3154  Default  1654.000000        file  1654.000000  26.0000  26.0000
4498  Default  1219.000000         key  1219.000000  25.0000  25.0000
5925  Default  4064.000000      people  4064.000000  24.0000  24.0000
4544  Default  3468.000000        know  3468.000000  23.0000  23.0000
2560  Default  1116.000000       drive  1116.000000  22.0000  22.0000
8762  Default   847.000000     version   847.000000  21.0000  21.0000
1157  Default   785.000000         car   785.000000  20.0000  20.0000
2509  Default  3881.000000         don  3881.000000  19.0000  19.0000
1506  Default  1429.000000         com  1429.000000  18.0000  18.0000
8676  Default  3161.000000         use  3161.000000  17.0000  17.0000
8983  Default  1206.000000     windows  1206.000000  16.0000  16.0000
3406  Default   806.000000        game   806.000000  15.0000  15.0000
8253  Default  1251.000000      thanks  1251.000000  14.0000  14.0000
1161  Default   803.000000        card   803.000000  13.0000  13.0000
4621  Default  1054.000000         law  1054.000000  12.0000  12.0000
8168  Default   800.000000        team   800.000000  11.0000  11.0000
4741  Default  3948.000000        like  3948.000000  10.0000  10.0000
6363  Default  1366.000000     program  1366.000000   9.0000   9.0000
4469  Default  3734.000000        just  3734.000000   8.0000   8.0000
4896  Default  1195.000000        mail  1195.000000   7.0000   7.0000
3552  Default  1216.000000  government  1216.000000   6.0000   6.0000
5453  Default  2610.000000         new  2610.000000   5.0000   5.0000
4397  Default   861.000000       jesus   861.000000   4.0000   4.0000
6193  Default  1070.000000       power  1070.000000   3.0000   3.0000
2493  Default  2744.000000        does  2744.000000   2.0000   2.0000
2517  Default   680.000000         dos   680.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
804   Topic20   447.864771         bhj   448.807768   4.6742  -2.8346
3498  Topic20   412.524276         giz   413.467273   4.6740  -2.9168
1083  Topic20   208.996424         bxn   209.939420   4.6718  -3.5968
6508  Topic20   126.370615         qax   127.313611   4.6688  -4.0999
5529  Topic20   111.876508        nrhj   112.819505   4.6679  -4.2217
301   Topic20   103.018035         amp   103.961032   4.6672  -4.3042
848   Topic20    97.758867         biz 

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
6      12.691766        1       1   59.502209   98.509978
19     11.686348        1       2  154.338110  -63.208879
8       8.568093        1       3  254.189149  -93.737686
15      7.682942        1       4 -220.650136  121.962184
14      7.401868        1       5   84.796720 -171.310909
0       6.705742        1       6   59.564838  -76.706938
13      6.569009        1       7  -29.359787   48.235251
4       5.887326        1       8  -16.262090 -145.771574
10      4.939733        1       9  -17.193046  159.354857
16      4.438698        1      10 -121.376111    7.731875
5       4.292685        1      11    3.181191 -391.492346
1       4.085641        1      12 -108.493506  113.108451
2       3.037200        1      13  157.656170  126.218036
9       2.451119        1      14  -15.391360  311.933615
17      2.436387        1      15  160.915144   31.884328
7       1.891398        1      16   56.685078    9.417807
3       1.511184        1      17 -108.479628 -176.585674
11      1.445251        1      18  -38.241361  -50.477341
12      1.346250        1      19   92.050970  207.971617
18      0.931361        1      20 -184.653499  -88.009886, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
5017  Default  4649.000000         max  4649.000000  30.0000  30.0000
2653  Default  2384.000000         edu  2384.000000  29.0000  29.0000
3524  Default  2037.000000         god  2037.000000  28.0000  28.0000
7678  Default  1274.000000       space  1274.000000  27.0000  27.0000
3154  Default  1654.000000        file  1654.000000  26.0000  26.0000
4498  Default  1219.000000         key  1219.000000  25.0000  25.0000
5925  Default  4064.000000      people  4064.000000  24.0000  24.0000
4544  Default  3468.000000        know  3468.000000  23.0000  23.0000
2560  Default  1116.000000       drive  1116.000000  22.0000  22.0000
8762  Default   847.000000     version   847.000000  21.0000  21.0000
1157  Default   785.000000         car   785.000000  20.0000  20.0000
2509  Default  3881.000000         don  3881.000000  19.0000  19.0000
1506  Default  1429.000000         com  1429.000000  18.0000  18.0000
8676  Default  3161.000000         use  3161.000000  17.0000  17.0000
8983  Default  1206.000000     windows  1206.000000  16.0000  16.0000
3406  Default   806.000000        game   806.000000  15.0000  15.0000
8253  Default  1251.000000      thanks  1251.000000  14.0000  14.0000
1161  Default   803.000000        card   803.000000  13.0000  13.0000
4621  Default  1054.000000         law  1054.000000  12.0000  12.0000
8168  Default   800.000000        team   800.000000  11.0000  11.0000
4741  Default  3948.000000        like  3948.000000  10.0000  10.0000
6363  Default  1366.000000     program  1366.000000   9.0000   9.0000
4469  Default  3734.000000        just  3734.000000   8.0000   8.0000
4896  Default  1195.000000        mail  1195.000000   7.0000   7.0000
3552  Default  1216.000000  government  1216.000000   6.0000   6.0000
5453  Default  2610.000000         new  2610.000000   5.0000   5.0000
4397  Default   861.000000       jesus   861.000000   4.0000   4.0000
6193  Default  1070.000000       power  1070.000000   3.0000   3.0000
2493  Default  2744.000000        does  2744.000000   2.0000   2.0000
2517  Default   680.000000         dos   680.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
804   Topic20   447.864771         bhj   448.807768   4.6742  -2.8346
3498  Topic20   412.524276         giz   413.467273   4.6740  -2.9168
1083  Topic20   208.996424         bxn   209.939420   4.6718  -3.5968
6508  Topic20   126.370615         qax   127.313611   4.6688  -4.0999
5529  Topic20   111.876508        nrhj   112.819505   4.6679  -4.2217
301   Topic20   103.0